In [1]:
import torch

torch.cuda.is_available()

C:\Users\Angie Krowicki.DESKTOP-FC60OA3\miniconda3\envs\gym\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
torch. __version__

'1.12.0'

In [3]:
!git clone https://github.com/yl4579/StarGANv2-VC.git

Cloning into 'StarGANv2-VC'...
Checking out files:  78% (26/33)
Checking out files:  81% (27/33)
Checking out files:  84% (28/33)
Checking out files:  87% (29/33)
Checking out files:  90% (30/33)
Checking out files:  93% (31/33)
Checking out files:  96% (32/33)
Checking out files: 100% (33/33)
Checking out files: 100% (33/33), done.


In [1]:
%cd StarGANv2-VC

C:\Users\Angie Krowicki.DESKTOP-FC60OA3\Desktop\projects\scrape\conversion\StarGANv2-VC


In [5]:
!pip install munch parallel_wavegan pydub pyyaml click librosa

  Using cached SoundFile-0.10.3.post1-py2.py3.cp26.cp27.cp32.cp33.cp34.cp35.cp36.pp27.pp32.pp33-none-win_amd64.whl (689 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached audioread-2.1.9-py3-none-any.whl
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Created wheel for parallel-wavegan: filename=parallel_wavegan-0.5.5-py3-none-any.whl size=72465 sha256=8336f08a3e21a97ff4f7da3bab2d377da8b030c20f77be20e7fe4c17474f013d
  Stored in directory: c:\users\angie krowicki.desktop-fc60oa3\appdata\local\pip\cache\wheels\67\cb\04\5bdedaaec4d6bf6dd46df360c30515ae58364cfa48e2f953bb
  Created wheel for kaldiio: filename=kaldiio-2.17.2-py3-none-any.whl size=24

In [5]:
!pip install cffi==1.5.0 --user

In [2]:
# load packages
import random
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
import librosa

from Utils.ASR.models import ASRCNN
from Utils.JDC.model import JDCNet
from models import Generator, MappingNetwork, StyleEncoder

%matplotlib inline

C:\Users\Angie Krowicki.DESKTOP-FC60OA3\miniconda3\envs\gym\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Source: http://speech.ee.ntu.edu.tw/~jjery2243542/resource/model/is18/en_speaker_used.txt
# Source: https://github.com/jjery2243542/voice_conversion

speakers = [225,228,229,230,231,233,236,239,240,244,226,227,232,243,254,256,258,259,270,273]

to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300)
mean, std = -4, 4

def preprocess(wave):
    wave_tensor = torch.from_numpy(wave).float()
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor

def build_model(model_params={}):
    args = Munch(model_params)
    generator = Generator(args.dim_in, args.style_dim, args.max_conv_dim, w_hpf=args.w_hpf, F0_channel=args.F0_channel)
    mapping_network = MappingNetwork(args.latent_dim, args.style_dim, args.num_domains, hidden_dim=args.max_conv_dim)
    style_encoder = StyleEncoder(args.dim_in, args.style_dim, args.num_domains, args.max_conv_dim)
    
    nets_ema = Munch(generator=generator,
                     mapping_network=mapping_network,
                     style_encoder=style_encoder)

    return nets_ema

def compute_style(speaker_dicts):
    reference_embeddings = {}
    for key, (path, speaker) in speaker_dicts.items():
        if path == "":
            label = torch.LongTensor([speaker]).to('cuda')
            latent_dim = starganv2.mapping_network.shared[0].in_features
            ref = starganv2.mapping_network(torch.randn(1, latent_dim).to('cuda'), label)
        else:
            wave, sr = librosa.load(path, sr=24000)
            audio, index = librosa.effects.trim(wave, top_db=30)
            if sr != 24000:
                wave = librosa.resample(wave, sr, 24000)
            mel_tensor = preprocess(wave).to('cuda')

            with torch.no_grad():
                label = torch.LongTensor([speaker])
                ref = starganv2.style_encoder(mel_tensor.unsqueeze(1), label)
        reference_embeddings[key] = (ref, label)
    
    return reference_embeddings

In [4]:
# load F0 model

F0_model = JDCNet(num_class=1, seq_len=192)
params = torch.load("Utils/JDC/bst.t7")['net']
F0_model.load_state_dict(params)
_ = F0_model.eval()
F0_model = F0_model.to('cuda')

In [6]:
# load vocoder
from parallel_wavegan.utils import load_model
vocoder = load_model("Vocoder/checkpoint-400000steps.pkl").to('cuda').eval()
vocoder.remove_weight_norm()
_ = vocoder.eval()

In [7]:
# load starganv2

model_path = 'Models/epoch_00150.pth'

with open('Models/config.yml') as f:
    starganv2_config = yaml.safe_load(f)
starganv2 = build_model(model_params=starganv2_config["model_params"])
params = torch.load(model_path, map_location='cpu')
params = params['model_ema']
_ = [starganv2[key].load_state_dict(params[key]) for key in starganv2]
_ = [starganv2[key].eval() for key in starganv2]
starganv2.style_encoder = starganv2.style_encoder.to('cuda')
starganv2.mapping_network = starganv2.mapping_network.to('cuda')
starganv2.generator = starganv2.generator.to('cuda')

In [13]:
#@title use

# load input wave
selected_speakers = [243]
k = random.choice(selected_speakers)
wav_path = 'Demo/VCTK-corpus/p' + str(k) + '/p' + str(k) + '_023.wav'
audio, source_sr = librosa.load("C:/Users/Angie Krowicki.DESKTOP-FC60OA3/Downloads/vo-[AudioTrimmer.com]-fix.wav", sr=22050)
#audio = audio / np.max(np.abs(audio))
audio.dtype = np.float32

# with reference, using style encoder
speaker_dicts = {}
for s in selected_speakers:
    k = s
    speaker_dicts['p' + str(s)] = ('Demo/VCTK-corpus/p' + str(k) + '/p' + str(k) + '_023.wav', speakers.index(s))

reference_embeddings = compute_style(speaker_dicts)

# conversion 
import time
start = time.time()
    
source = preprocess(audio).to('cuda:0')
keys = []
converted_samples = {}
reconstructed_samples = {}
converted_mels = {}

for key, (ref, _) in reference_embeddings.items():
    with torch.no_grad():
        f0_feat = F0_model.get_feature_GAN(source.unsqueeze(1))
        out = starganv2.generator(source.unsqueeze(1), ref, F0=f0_feat)
        
        c = out.transpose(-1, -2).squeeze().to('cuda')
        y_out = vocoder.inference(c)
        y_out = y_out.view(-1).cpu()

        if key not in speaker_dicts or speaker_dicts[key][0] == "":
            recon = None
        else:
            wave, sr = librosa.load(speaker_dicts[key][0], sr=24000)
            mel = preprocess(wave)
            c = mel.transpose(-1, -2).squeeze().to('cuda')
            recon = vocoder.inference(c)
            recon = recon.view(-1).cpu().numpy()

    converted_samples[key] = y_out.numpy()
    reconstructed_samples[key] = recon

    converted_mels[key] = out
    
    keys.append(key)
end = time.time()
print('total processing time: %.3f sec' % (end - start) )

import IPython.display as ipd
for key, wave in converted_samples.items():
    print('Converted: %s' % key)
    display(ipd.Audio(wave, rate=24000))

total processing time: 1.316 sec
Converted: p243


In [11]:
!pip install sounddevice wavio scipy

In [18]:
import sounddevice as sd
from scipy.io.wavfile import write
import wavio as wv
  
# Sampling frequency
freq = 22050 # minimum frequency that keeps lots of data
  
# Recording duration
duration = 0.5
  




def convert():
    # conversion 
    speaker_dicts = {}
    selected_speakers = [273]
    for s in selected_speakers:
        k = s
        speaker_dicts['p' + str(s)] = ('', speakers.index(s))

    reference_embeddings = compute_style(speaker_dicts)

    import time
    start = time.time()
    audio, source_sr = librosa.load("recdata.wav", sr=22050)
    audio = audio / np.max(np.abs(audio))
    audio.dtype = np.float32
    source = preprocess(audio).to('cuda:0')
    keys = []
    converted_samples = {}
    reconstructed_samples = {}
    converted_mels = {}

    for key, (ref, _) in reference_embeddings.items():
        with torch.no_grad():
            f0_feat = F0_model.get_feature_GAN(source.unsqueeze(1))
            out = starganv2.generator(source.unsqueeze(1), ref, F0=f0_feat)
        
            c = out.transpose(-1, -2).squeeze().to('cuda')
            y_out = vocoder.inference(c)
            y_out = y_out.view(-1).cpu()

            if key not in speaker_dicts or speaker_dicts[key][0] == "":
                recon = None
            else:
                wave, sr = librosa.load(speaker_dicts[key][0], sr=24000)
                mel = preprocess(wave)
                c = mel.transpose(-1, -2).squeeze().to('cuda')
                recon = vocoder.inference(c)
                recon = recon.view(-1).cpu().numpy()

        converted_samples[key] = y_out.numpy()
        reconstructed_samples[key] = recon

        converted_mels[key] = out
    
        keys.append(key)
    end = time.time()
    print('total processing time: %.3f sec' % (end - start) )
    for key, wave in converted_samples.items():
        display(ipd.Audio(wave, rate=24000, autoplay=True))
    return


while True:
    # Start recorder with the given values 
    # of duration and sample frequency
    recording = sd.rec(int(duration * freq), 
                   samplerate=freq, channels=1)
  
    # Record audio for the given number of seconds
    sd.wait()
  
    # This will convert the NumPy array to an audio
    # file with the given sampling frequency
    write("recdata.wav", freq, recording)
    convert()



total processing time: 0.090 sec


total processing time: 0.073 sec


total processing time: 0.078 sec


total processing time: 0.085 sec


total processing time: 0.073 sec


...

total processing time: 0.076 sec


total processing time: 0.074 sec


total processing time: 0.072 sec


total processing time: 0.075 sec


KeyboardInterrupt: 

In [44]:
import threading
import sounddevice as sd
from scipy.io.wavfile import write
import wavio as wv
import time
  
# Sampling frequency
freq = 22050 # minimum frequency that keeps lots of data
  
# Recording duration
duration = 2
  
def record():
    # Start recorder with the given values 
    # of duration and sample frequency
    recording = sd.rec(int(duration * freq), 
                   samplerate=freq, channels=1, dtype=np.float32)
  
    # Record audio for the given number of seconds
    sd.wait()
  
    # This will convert the NumPy array to an audio
    # file with the given sampling frequency
    write("recdata.wav", freq, recording)

# conversion 
speaker_dicts = {}
selected_speakers = [273]
for s in selected_speakers:
    k = s
    speaker_dicts['p' + str(s)] = ('', speakers.index(s))
global reference_embeddings
reference_embeddings = compute_style(speaker_dicts)

    
def convert():
    time.sleep(1)
    global reference_embeddings
    start = time.time()
    audio, source_sr = librosa.load("recdata.wav", sr=22050)
    audio = audio / np.max(np.abs(audio))
    audio.dtype = np.float32
    source = preprocess(audio).to('cuda:0')
    keys = []
    converted_samples = {}
    reconstructed_samples = {}
    converted_mels = {}

    for key, (ref, _) in reference_embeddings.items():
        with torch.no_grad():
            f0_feat = F0_model.get_feature_GAN(source.unsqueeze(1))
            out = starganv2.generator(source.unsqueeze(1), ref, F0=f0_feat)
        
            c = out.transpose(-1, -2).squeeze().to('cuda')
            y_out = vocoder.inference(c)
            y_out = y_out.view(-1).cpu()

            if key not in speaker_dicts or speaker_dicts[key][0] == "":
                recon = None
            else:
                wave, sr = librosa.load(speaker_dicts[key][0], sr=24000)
                mel = preprocess(wave)
                c = mel.transpose(-1, -2).squeeze().to('cuda')
                recon = vocoder.inference(c)
                recon = recon.view(-1).cpu().numpy()

        converted_samples[key] = y_out.numpy()
        reconstructed_samples[key] = recon

        converted_mels[key] = out
    
        keys.append(key)
    end = time.time()
    print('total processing time: %.3f sec' % (end - start) )
    key, wave = list(converted_samples.items())[0]
    display(ipd.Audio(wave, rate=24000, autoplay=True))
    return wave

while True:
    threading.Thread(target=record).start()
    wave = convert()
    #sd.play(wave, 24000)

C:\Users\Angie Krowicki.DESKTOP-FC60OA3\miniconda3\envs\gym\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  app.launch_new_instance()


total processing time: 0.211 sec


total processing time: 0.125 sec


total processing time: 0.124 sec


total processing time: 0.135 sec


...

total processing time: 0.143 sec


total processing time: 0.157 sec


total processing time: 0.169 sec


total processing time: 0.155 sec


KeyboardInterrupt: 